In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D 
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D 
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_uniform
from resnet import ResnetBuilder
from resnet_builder import resnet
import os
# from resnet_builder import resnet
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
ROOT_DIR = os.path.abspath("../data")
ROOT_DIR
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TRAIN_DIR
TEST_DIR = os.path.join(ROOT_DIR,"test")

In [3]:
TEST_DIR

'C:\\Users\\cptien\\Desktop\\ML100-Days\\data\\test'

In [4]:
aug_gen = ImageDataGenerator(rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest'
                            )

In [5]:
# Image Generator
train_datagen = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Misc Information
num_classes = 5
batch_size = 32


In [46]:
def export_results(train, test, model, output_name):
    test.reset()
    pred = model.predict_generator(test, steps=len(test),verbose=1)
    predicted_class_indices= np.argmax(pred, axis=1)
    
    labels = (train.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]
    
    test_path=test.filenames
    filenames=[]
    for n in test_path:
        seg1, seg2 = n.split("/")
        front, back = seg2.split('.')

        filenames.append(front)

    results= pd.DataFrame({"id":filenames,
                      "flower_class":predicted_class_indices})
    results.to_csv("./submission"+str(output_name)+".csv",index=False)

In [6]:
# Load training set
train = train_datagen.flow_from_directory(TRAIN_DIR,
                                          target_size=(64,64), 
                                          batch_size=32, 
                                          class_mode='categorical', 
                                          shuffle=True,
                                         )
total_samples = train.n
# Load testing set
test = test_datagen.flow_from_directory(TEST_DIR,
                                        target_size=(64,64),
                                        batch_size=1, 
                                        class_mode=None,
                                        shuffle=False)
# train = keras.utils.to_categorical(train, 5)
# test = keras.utils.to_categorical(test,5)

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'C:\\Users\\cptien\\Desktop\\ML100-Days\\data\\train'

In [15]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau


earlystop = EarlyStopping(monitor="loss", 
                          patience=10, 
                          verbose=1
                          )

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='loss', 
                              patience=5, 
                              verbose=1)

In [57]:
# Use resnet 50
clf = ResnetBuilder.build_resnet_50(input_shape=(3,64,64), num_outputs=num_classes)
clf.summary()
clf.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
clf.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 100,
                  verbose=1,
                  shuffle=True,
                  callbacks=[earlystop, reduce_lr]
                  )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_443 (Conv2D)             (None, 32, 32, 64)   9472        input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_407 (BatchN (None, 32, 32, 64)   256         conv2d_443[0][0]                 
__________________________________________________________________________________________________
activation_407 (Activation)     (None, 32, 32, 64)   0           batch_normalization_407[0][0]    
__________________________________________________________________________________________________
max_poolin

                                                                 conv2d_482[0][0]                 
__________________________________________________________________________________________________
batch_normalization_443 (BatchN (None, 4, 4, 1024)   4096        add_144[0][0]                    
__________________________________________________________________________________________________
activation_443 (Activation)     (None, 4, 4, 1024)   0           batch_normalization_443[0][0]    
__________________________________________________________________________________________________
conv2d_483 (Conv2D)             (None, 4, 4, 256)    262400      activation_443[0][0]             
__________________________________________________________________________________________________
batch_normalization_444 (BatchN (None, 4, 4, 256)    1024        conv2d_483[0][0]                 
__________________________________________________________________________________________________
activation

88/88 [==============================] - 316s 4s/step - loss: 6.3838 - acc: 0.4740
Epoch 2/100
88/88 [==============================] - 254s 3s/step - loss: 5.1265 - acc: 0.5445
Epoch 3/100
88/88 [==============================] - 246s 3s/step - loss: 4.2032 - acc: 0.5987
Epoch 4/100
88/88 [==============================] - 247s 3s/step - loss: 3.5585 - acc: 0.6086
Epoch 5/100
88/88 [==============================] - 245s 3s/step - loss: 3.0523 - acc: 0.6239
Epoch 6/100
88/88 [==============================] - 242s 3s/step - loss: 2.7236 - acc: 0.6267
Epoch 7/100
88/88 [==============================] - 239s 3s/step - loss: 2.4696 - acc: 0.6249
Epoch 8/100
88/88 [==============================] - 239s 3s/step - loss: 2.1907 - acc: 0.6682
Epoch 9/100
88/88 [==============================] - 236s 3s/step - loss: 2.0089 - acc: 0.6851
Epoch 10/100
88/88 [==============================] - 236s 3s/step - loss: 1.8940 - acc: 0.6818
Epoch 11/100
88/88 [==============================] - 235s 3s

In [58]:
export_results(train,test, clf, 5)

2000/2000 [==============================] - 213s 106ms/step


In [56]:
# Transfer learning using resnet
model_resnet = resnet(input_shape=(64,64,3), num_classes=num_classes)
model_resnet.summary()
model_resnet.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
model_resnet.fit_generator(train,
                  steps_per_epoch = 1, #int(total_samples/batch_size),
                  epochs = 2, # change from 50
                  verbose=1,
                  shuffle=True,
                  callbacks=[earlystop, reduce_lr]
                  )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_412 (Conv2D)             (None, 64, 64, 16)   448         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_379 (BatchN (None, 64, 64, 16)   64          conv2d_412[0][0]                 
__________________________________________________________________________________________________
activation_379 (Activation)     (None, 64, 64, 16)   0           batch_normalization_379[0][0]    
__________________________________________________________________________________________________
conv2d_413

KeyboardInterrupt: 

In [ ]:
export_results(train,test, model_resnet, 2222)

In [ ]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("/")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission4.csv", index=False)

In [18]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("/")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission4.csv", index=False)

In [42]:
emp = pd.DataFrame()
a=2

In [43]:
emp.to_csv("./test"+str(a)+".csv")

In [172]:
#Build Model

clf = Sequential()
clf.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=(64,64,3)))
clf.add(Activation('relu'))
clf.add(Conv2D(32, (3, 3)))
clf.add(Activation('relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
clf.add(Dropout(0.25))

clf.add(Conv2D(64, (3, 3), padding='same'))
clf.add(Activation('relu'))
clf.add(Conv2D(64, (3, 3)))
clf.add(Activation('relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
clf.add(Dropout(0.25))

clf.add(Flatten())
clf.add(Dense(512))
clf.add(Activation('relu'))
clf.add(Dropout(0.5))
clf.add(Dense(num_classes))
clf.add(Activation('softmax'))
clf.summary()
clf.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_166 (Conv2D)          (None, 64, 64, 64)        1792      
_________________________________________________________________
activation_148 (Activation)  (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_167 (Conv2D)          (None, 62, 62, 32)        18464     
_________________________________________________________________
activation_149 (Activation)  (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_168 (Conv2D)          (None, 31, 31, 64)        18496     
__________

In [173]:
clf.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 50,
                  verbose=1,
                  shuffle=True
                  )

Epoch 1/50
88/88 [==============================] - 89s 1s/step - loss: 1.5526 - acc: 0.3261
Epoch 2/50
88/88 [==============================] - 91s 1s/step - loss: 1.2736 - acc: 0.4534
Epoch 3/50
88/88 [==============================] - 85s 971ms/step - loss: 1.1690 - acc: 0.5303
Epoch 4/50
88/88 [==============================] - 86s 975ms/step - loss: 1.1042 - acc: 0.5664
Epoch 5/50
88/88 [==============================] - 87s 992ms/step - loss: 1.0144 - acc: 0.6026
Epoch 6/50
88/88 [==============================] - 86s 973ms/step - loss: 1.0234 - acc: 0.6079
Epoch 7/50
88/88 [==============================] - 86s 974ms/step - loss: 0.9508 - acc: 0.6397
Epoch 8/50
88/88 [==============================] - 85s 971ms/step - loss: 0.9394 - acc: 0.6546
Epoch 9/50
88/88 [==============================] - 86s 979ms/step - loss: 0.9315 - acc: 0.6446
Epoch 10/50
88/88 [==============================] - 88s 1s/step - loss: 0.9080 - acc: 0.6486
Epoch 11/50
88/88 [=============================

KeyboardInterrupt: 

In [107]:
labels = (train.class_indices)
labels = dict((n,l) for l,n in labels.items())
labels

{0: 'daisy', 1: 'dandelion', 2: 'rose', 3: 'sunflower', 4: 'tulip'}

In [174]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
clf.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 


Saved model to disk


In [176]:
test.reset()
pred = clf.predict_generator(test, steps=len(test),verbose=1)
predicted_class_indices=np.argmax(pred, axis=1)


2000/2000 [==============================] - 45s 22ms/step


In [177]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("\\")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission2.csv", index=False)

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [84]:
res_model = ResNet50()
res_model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
res_model.fit_generator(train,
                  steps_per_epoch = int(len(train)/32),
                  epochs = 25,
                  verbose=1,
                  shuffle=True
                  )

NameError: name 'convolutional_block' is not defined

In [18]:
ROOT_DIR

'C:\\Users\\cptien\\Desktop\\data'